# 2D NMR plotting

In [20]:
import matplotlib
#matplotlib.use('nbagg') # this allows having inline graph, which can be modified (i.e., zooming)

** Warning **
The above command does work only if this notebook is within a brower.

The following command must be executed in a terminal window at the prompt:

> cd '/the/path/where/your/notebooks/are/saved'

> ipython notebook

which should automatically open a browser. 

Then the present notebook can be open from the file browser.

##### Python imports

In [21]:
import os

from matplotlib import rcParams
from matplotlib import pyplot as pl

import numpy as np
import nmrglue as ng
import pandas as pd

##### Local imports

In [22]:
from lcslib.nmr.bruker import read_pdata
from lcslib.nmr.units import scale
from lcslib.nmr.plotting import make_fig_2D

##### Graph setup

In [23]:
print pl.style.available  
pl.style.use('lcsdefault')

[u'seaborn-darkgrid', u'seaborn-notebook', u'classic', u'seaborn-ticks', u'grayscale', u'bmh', u'seaborn-talk', u'dark_background', u'ggplot', u'fivethirtyeight', u'lcsdefault', u'seaborn-colorblind', u'seaborn-deep', u'seaborn-whitegrid', u'seaborn-bright', u'seaborn-poster', u'seaborn-muted', u'seaborn-paper', u'seaborn-white', u'seaborn-pastel', u'seaborn-dark', u'lcsgray', u'seaborn-dark-palette']


##### i/o directory

In [24]:
OUTPUTDIR = 'figures'
INPUTDIR = os.path.expanduser(os.path.join('~','Documents','Aurelie','data','eddy','nmr'))

##### import data

In [25]:
# 2D
path = os.path.join(INPUTDIR,'151202')
expno = os.path.join(path,'12')
procno = os.path.join(expno, 'pdata','1')

dic, data = read_pdata(procno, all_components=True) # read specs and proc
                            #data (all_components=True to read all spectra)

# for 2D spectra : data = [2rr, 2ri, 2ir, 2ii]
rr, ri, ir, ii = data

# make pd.DataFrame
x = scale(dic, unit='ppm', axis=-1)
y = scale(dic, unit='ppm', axis=0) 
d2rr = pd.DataFrame(rr.astype(np.float64, copy=False), index=y, columns=x)
#d2rr

In [26]:
# MAS
procnomas= os.path.join(path,'10','pdata','1')

dicm, datam = read_pdata(procnomas, all_components=True) # read specs and proc
                            #data (all_components=True to read all spectra) useful for rephasing

# for 1D spectra : data = [1r, 1i]
r, i = datam

# make pd.Series
xm = scale(dicm, unit='ppm', axis=-1)
d1r = pd.Series(r.astype(np.float64, copy=False), index=xm)
#d1r

##### Plot

In [42]:
PROJECTION = True
GRAYSCALE = False

matplotlib.rcdefaults()
pl.style.use('lcsdefault')
if GRAYSCALE:
    pl.style.use('lcsgray')

if not PROJECTION:
    ax = make_fig_2D(d2rr, 
                proj= True,
                projtype='sum',
                diagonal= 2,
                xlim = (20,-5),
                ylim = (30,-5),
                ylabel = r'$\mathrm{\delta\, _{DQ}\,/ppm}$',
                xlabel = r'$\mathrm{\delta\, _{SQ}\,/ppm}$',
                clstart = 0, # 'min'=-1
                clspacing=250,
                clexponent=1.3,
                clw=0.7,
                #cmap='Wistia', # adapted for deuteranope blind-people
                figname = os.path.join(OUTPUTDIR,'2d_1HBE.png'),
                left=0.2)
else:
    
    xlim = (20,-5)
    ylim = (30,-5)
    ax, axx, axy = make_fig_2D(d2rr, 
                mas=d1r/3+1.0e8,
                proj= True,
                diagonal= 2,
                xlim = xlim,
                ylim = ylim,
                ylabel = r'$\mathrm{\delta\, _{DQ}\,/ppm}$',
                xlabel = r'$\mathrm{\delta\, _{SQ}\,/ppm}$',
                clstart = 0, # if you want to start at min: use -1
                clspacing=250,
                clexponent=1.3,
                clw=0.7,
                figname = os.path.join(OUTPUTDIR, 'figureBABA_BEA_MFI.png')
                )
    
    # legends (#TODO improve this)
    dqfiltline, masline = axx.get_lines()
    dqline, = axy.get_lines()
    scl = axx.get_ylim()
    off = (scl[1]-scl[0])*.1 
    axx.text(xlim[0], dqfiltline.get_ydata()[-1]+off, 'DQ (projection)', color=dqfiltline.get_color(), fontsize=8) 
    axx.text(xlim[0], masline.get_ydata()[-1]+off, 'MAS', color=axx.get_lines()[1].get_color(), fontsize=8) 
    axy.text(dqline.get_xdata()[-1]+off*.75, ylim[0], 'SQ (projection)', color=axy.get_lines()[0].get_color(), 
                                                         rotation = -90, fontsize=8, va='bottom')
#--------
pl.show()
